In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Funs

In [35]:
def prep_data(sim):
    X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
    y = np.squeeze(np.stack([src.data for src in sim.source_data]))
    for i, (x_sample, y_sample) in enumerate(zip(X, y)):
        # X[i] = np.stack([(x - np.mean(x)) / np.std(x) for x in x_sample.T], axis=0).T
        # y[i] = np.stack([ y / np.max(abs(y)) for y in y_sample.T], axis=0).T

        X[i] = np.stack([x - np.mean(x) for x in x_sample.T], axis=0).T
        X[i] /= X[i].std()
        y[i] /= np.max(abs(y[i]))

    X = np.swapaxes(X, 1,2)
    y = np.swapaxes(y, 1,2)
    
    return X, y
def make_mask(y, thresh=0.001):
    y_mask = np.zeros((y.shape[0], n_dipoles))
    for i, y_samp in enumerate(y):
        yy = abs(y_samp).mean(axis=0)
        

        y_mask[i] = (yy > yy.max()*thresh).astype(int)
    return y_mask


# Forward

In [3]:
info = get_info(kind='biosemi128')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.5s remaining:    3.5s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    3.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished


# Simulation

In [46]:
settings = dict(number_of_sources=(1, 15), extents=(1, 40), duration_of_trial=0.01, target_snr=1e99)

sim = Simulation(fwd, info, settings).simulate(10000)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

stc.data /= abs(stc.data).max()
brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)
evoked.plot_joint()

X, y = prep_data(sim)
y_mask = make_mask(y, thresh=0.001)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 10000/10000 [00:01<00:00, 5493.97it/s]


source data shape:  (1284, 10) (1284, 10)


100%|██████████| 10000/10000 [00:58<00:00, 170.34it/s]


Using control points [0.08123995 0.12730579 0.66833728]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


In [47]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Bidirectional, LSTM, multiply, Dropout
from tensorflow.keras import backend as K

n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 300
n_lstm_units = 64
activation_function = "tanh"
batch_size = 32
epochs = 30
dropout = 0.2

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')

fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function), 
            name='FC1')(inputs)

# Masking
lstm1 = Bidirectional(LSTM(n_lstm_units, return_sequences=False, 
            input_shape=(None, n_dense_units), dropout=dropout), 
            name='LSTM1')(fc1)
out = Dense(n_dipoles, 
            activation="softmax", 
            name='Mask')(lstm1)

model = tf.keras.Model(inputs=inputs, outputs=out, name='Prelocalizer')

model.compile(loss="binary_crossentropy", optimizer="adam")
model.summary()
model.fit(X, y_mask, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "Prelocalizer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, None, 128)]       0         
                                                                 
 FC1 (TimeDistributed)       (None, None, 300)         38700     
                                                                 
 LSTM1 (Bidirectional)       (None, 128)               186880    
                                                                 
 Mask (Dense)                (None, 1284)              165636    
                                                                 
Total params: 391,216
Trainable params: 391,216
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
266/266 [==============================] - 7s 15ms/step - loss: 0.2357 - val_loss: 0.1974
Epoch 2/30
266/266 [==============================] - 3s 10ms/step - loss: 0.19

In [51]:
settings = dict(number_of_sources=3, extents=(1, 25), duration_of_trial=0.01, target_snr=1e99)

sim_test = Simulation(fwd, info, settings).simulate(2)
stc = sim_test.source_data[0]
evoked = sim_test.eeg_data[0].average()

stc.data /= abs(stc.data).max()
brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)
evoked.plot_joint()

X_test, y_test = prep_data(sim_test)
y_mask_test = make_mask(y_test, thresh=0.001)

y_hat = model.predict(X_test)[0]
stc_ = stc.copy()
stc_.data *= 0
stc_.data[:, 0] = y_hat

brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, 'Predicted Mask', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 1002.46it/s]


source data shape:  (1284, 10) (1284, 10)


100%|██████████| 2/2 [00:00<00:00, 138.20it/s]

Using control points [0.         0.         0.40258414]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
1/1 [==============================] - 0s 21ms/step
Using control points [0.00016571 0.00031698 0.02303454]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [0.00459589 0.00733536 0.03026608]
Using control points [0.         0.         0.36870243]
